<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=fb151b8def3c35b2c82934a5163b9e4f798ce5321c9cdaeb8d94f807c328c46c
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-10 13:09:42
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.43 C
-------------------
Today PnL: 42.13 K (0.3%)
Current PnL: -21.37 L (-14.0%)
CY Booked + Current PnL: -8.72 L (-5.71%)
-------------------
Total profit:  2.28 L
Total loss:  -23.65 L
-------------------
Total Booked + Current PnL: 18.26 L (14.54%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.83%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 88.34 L (61.69%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.63%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-6.53,65.0,X-LC,6.76,233657.0,23291.0,5491.0,1.34,11.07,2.35,13.68,37.0,4.24,1.64,29.42,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-2.33,69.0,M-LC,11.55,175325.0,34485.0,10011.0,0.59,24.49,5.71,31.59,66.0,3.44,1.23,57.46,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,101.49,51.0,M-SC,2.01,86788.0,-13989.0,14086.0,-0.87,-13.88,16.23,0.09,245.0,-0.99,0.61,14.02,OX40N,NTT,DURABLES
33,ICICIGI,2252.93,-12.45,61.0,X-MC,6.30,150634.0,14501.0,16088.0,0.76,10.65,10.68,22.47,91.0,0.90,1.06,25.01,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-11.26,42.0,X-MC,3.08,87735.0,-2943.0,17854.0,1.60,-3.25,20.35,16.44,101.0,-0.16,0.62,16.28,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIGOPNTS,1408.0,115.40,71.0,M-SC,5.88,141124.0,-33435.0,33461.0,13.34,-19.15,23.71,0.02,221.0,-1.00,0.99,22.75,OX40N,NTT,PAINTS
61,SAIL,228.0,58.78,71.0,M-MC,10.06,250551.0,25589.0,142062.0,3.19,11.37,56.70,74.53,192.0,0.18,1.76,47.30,XY24,BTT,STEEL
19,COFFEEDAY,80.0,-47.69,42.0,L-SC,27.36,77107.0,-36442.0,76490.0,3.00,-32.09,99.20,35.27,268.0,-0.48,0.54,88.72,XR,NTT,HOTELS
39,INFY,2275.0,-17.09,61.0,X-LC,4.86,323334.0,10446.0,161247.0,2.79,3.34,49.87,54.87,3.0,0.06,2.27,11.90,X40,BTT,IT
47,LAOPALA,464.0,99.12,46.0,H-SC,3.79,91058.0,-40047.0,91295.0,2.22,-30.55,100.26,39.09,142.0,-0.44,0.64,17.27,AR,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SURYODAY,240.0,53.51,39.0,H-SC,14.56,140678.0,-38393.0,104116.0,-3.74,-21.44,74.01,36.71,135.0,-0.37,0.99,39.58,XR,NTT,BANKS
65,SHALBY,327.0,1061.08,37.0,M-SC,17.15,157872.0,-23271.0,68737.0,-3.27,-12.85,43.54,25.10,235.0,-0.34,1.11,26.52,XY24,NTT,HEALTHCARE
85,WHIRLPOOL,2270.0,-42.18,46.0,M-SC,7.05,119977.0,5179.0,88867.0,-2.68,4.51,74.07,81.92,223.0,0.06,0.84,42.03,XR,NTT,DURABLES
25,GREENPANEL,537.0,203.62,40.0,M-SC,5.25,135542.0,-47536.0,126515.0,-2.63,-25.96,93.34,43.14,230.0,-0.38,0.95,27.50,XY24,NTT,MISC
0,5PAISA,593.0,81.49,18.0,H-SC,15.80,122886.0,-95610.0,123795.0,-2.27,-43.76,100.74,12.90,153.0,-0.77,0.86,0.00,OX40N,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,101.49,51.0,M-SC,2.01,86788.0,-13989.0,14086.0,-0.87,-13.88,16.23,0.09,245.0,-0.99,0.61,14.02,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,46.0,H-SC,2.37,222336.0,-47331.0,83665.0,0.29,-17.55,37.63,13.47,138.0,-0.57,1.56,13.05,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,115.40,71.0,M-SC,5.88,141124.0,-33435.0,33461.0,13.34,-19.15,23.71,0.02,221.0,-1.00,0.99,22.75,OX40N,NTT,PAINTS
67,SIS,528.0,2040.93,50.0,H-SC,2.30,86394.0,-24638.0,48242.0,1.86,-22.19,55.84,21.26,156.0,-0.51,0.61,16.73,OX40N,NTT,MISC
17,CERA,9475.0,-25.54,38.0,H-SC,1.88,135826.0,-40077.0,82093.0,-1.53,-22.78,60.44,23.89,149.0,-0.49,0.96,17.30,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-2.33,69.0,M-LC,11.55,175325.0,34485.0,10011.0,0.59,24.49,5.71,31.59,66.0,3.44,1.23,57.46,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,58.78,71.0,M-MC,10.06,250551.0,25589.0,142062.0,3.19,11.37,56.70,74.53,192.0,0.18,1.76,47.30,XY24,BTT,STEEL
31,HINDZINC,730.22,29.16,53.0,M-LC,9.41,210801.0,5725.0,106855.0,2.13,2.79,50.69,54.89,52.0,0.05,1.48,27.56,X5K,ATH,METALS
79,UNIONBANK,163.00,-2.33,69.0,M-LC,11.55,175325.0,34485.0,10011.0,0.59,24.49,5.71,31.59,66.0,3.44,1.23,57.46,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-42.18,46.0,M-SC,7.05,119977.0,5179.0,88867.0,-2.68,4.51,74.07,81.92,223.0,0.06,0.84,42.03,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.89,49.0,H-SC,5.23,128901.0,5565.0,126065.0,-0.63,4.51,97.80,106.72,119.0,0.04,0.91,28.95,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-42.18,46.0,M-SC,7.05,119977.0,5179.0,88867.0,-2.68,4.51,74.07,81.92,223.0,0.06,0.84,42.03,XR,NTT,DURABLES
31,HINDZINC,730.22,29.16,53.0,M-LC,9.41,210801.0,5725.0,106855.0,2.13,2.79,50.69,54.89,52.0,0.05,1.48,27.56,X5K,ATH,METALS
36,INDIAMART,4810.62,-53.89,49.0,H-SC,5.23,128901.0,5565.0,126065.0,-0.63,4.51,97.80,106.72,119.0,0.04,0.91,28.95,AR,ATH,MISC
86,WIPRO,420.00,-15.15,46.0,M-LC,5.60,148812.0,-2133.0,111594.0,1.49,-1.41,74.99,72.51,53.0,-0.02,1.05,4.52,XR,NTT,IT
46,KPIGREEN,731.05,4.03,46.0,H-SC,9.48,123971.0,-1326.0,60250.0,-2.24,-1.06,48.60,47.03,141.0,-0.02,0.87,55.22,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,22.0,X-LC,34.23,147269.0,-129254.0,239327.0,-0.53,-46.74,162.51,39.81,54.0,-0.54,1.04,3.80,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.72,28.0,X-SC,8.15,87150.0,-41520.0,86819.0,-1.12,-32.27,99.62,35.20,219.0,-0.48,0.61,0.03,X40,NTT,FOOTWEAR
23,DMART,5391.45,-20.90,28.0,X-LC,7.17,201535.0,-8872.0,68038.0,0.49,-4.22,33.76,28.12,38.0,-0.13,1.42,18.41,X40N,ATH,FMCG
44,JSWINFRA,342.00,-23.96,30.0,X-MC,4.44,191059.0,-9288.0,39798.0,1.02,-4.64,20.83,15.23,178.0,-0.23,1.34,26.12,X40N,NTT,REALTY
30,HINDUNILVR,2922.00,-14.10,32.0,X-LC,6.23,241400.0,-15663.0,50791.0,0.00,-6.09,21.04,13.67,24.0,-0.31,1.70,13.96,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-31.60,34.0,X-MC,0.29,197910.0,-2910.0,70713.0,0.15,-1.45,35.73,33.76,80.0,-0.04,1.39,3.67,X40,ATH,FMCG
41,ITC,452.00,-38.34,44.0,X-LC,0.81,198436.0,-1702.0,22582.0,0.43,-0.85,11.38,10.44,4.0,-0.08,1.40,4.67,X40,NTT,FMCG
66,SIEMENS,4671.50,-5.18,40.0,H-LC,1.56,152545.0,-33550.0,81032.0,0.58,-18.03,53.12,25.51,15.0,-0.41,1.07,12.41,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.16,40.0,X-MC,3.27,306684.0,-21307.0,127826.0,0.82,-6.50,41.68,32.48,92.0,-0.17,2.16,4.19,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-7.38,59.0,X-LC,3.81,242675.0,8531.0,40090.0,-1.27,3.64,16.52,20.77,86.0,0.21,1.71,10.81,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.72,28.0,X-SC,8.15,87150.0,-41520.0,86819.0,-1.12,-32.27,99.62,35.20,219.0,-0.48,0.61,0.03,X40,NTT,FOOTWEAR
53,QUESS,986.00,-50.41,34.0,X-SC,37.34,49060.0,-15946.0,166873.0,-1.11,-24.53,340.14,232.18,198.0,-0.10,0.35,0.41,XY24,NTT,MISC
51,PAGEIND,51769.93,-28.87,39.0,X-MC,7.53,80350.0,-3160.0,23189.0,1.09,-3.78,28.86,23.98,82.0,-0.14,0.57,1.82,X40,ATH,APPARELS
1,ABB,7934.00,-42.59,33.0,H-LC,4.95,239784.0,-21835.0,141041.0,-0.42,-8.35,58.82,45.57,7.0,-0.15,1.69,2.03,AR,NTT,ELECTRICAL
20,COLPAL,3726.84,-6.74,41.0,X-MC,7.18,217550.0,-45815.0,155135.0,0.38,-17.40,71.31,41.51,84.0,-0.30,1.53,2.09,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.41,34.0,X-SC,37.34,49060.0,-15946.0,166873.0,-1.11,-24.53,340.14,232.18,198.0,-0.10,0.35,0.41,XY24,NTT,MISC
51,PAGEIND,51769.93,-28.87,39.0,X-MC,7.53,80350.0,-3160.0,23189.0,1.09,-3.78,28.86,23.98,82.0,-0.14,0.57,1.82,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.85,40.0,X-SC,4.21,80624.0,-64536.0,140463.0,-0.60,-44.46,174.22,52.31,136.0,-0.46,0.57,7.92,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.72,28.0,X-SC,8.15,87150.0,-41520.0,86819.0,-1.12,-32.27,99.62,35.20,219.0,-0.48,0.61,0.03,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-11.26,42.0,X-MC,3.08,87735.0,-2943.0,17854.0,1.60,-3.25,20.35,16.44,101.0,-0.16,0.62,16.28,X40,ATH,PHARMA


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-28.69,51.0,X-LC,9.56,284482.0,-61474.0,128188.0,1.16,-17.77,45.06,19.28,1.0,-0.48,2.00,5.17,X40,ATH,IT
39,INFY,2275.00,-17.09,61.0,X-LC,4.86,323334.0,10446.0,161247.0,2.79,3.34,49.87,54.87,3.0,0.06,2.27,11.90,X40,BTT,IT
41,ITC,452.00,-38.34,44.0,X-LC,0.81,198436.0,-1702.0,22582.0,0.43,-0.85,11.38,10.44,4.0,-0.08,1.40,4.67,X40,NTT,FMCG
83,VBL,671.64,-16.88,52.0,X-LC,5.64,299222.0,-16620.0,129294.0,-0.26,-5.26,43.21,35.67,5.0,-0.13,2.11,7.86,X40N,ATH,FMCG
1,ABB,7934.00,-42.59,33.0,H-LC,4.95,239784.0,-21835.0,141041.0,-0.42,-8.35,58.82,45.57,7.0,-0.15,1.69,2.03,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,42.0,L-SC,27.36,77107.0,-36442.0,76490.0,3.00,-32.09,99.20,35.27,268.0,-0.48,0.54,88.72,XR,NTT,HOTELS
7,ASIANTILES,137.00,7394.44,64.0,L-SC,13.01,82204.0,-11606.0,88221.0,0.53,-12.37,107.32,81.67,269.0,-0.13,0.58,59.19,XR,NTT,CERAMICS
49,MASFIN,398.61,-19.15,50.0,H-SC,7.97,92100.0,-5880.0,27483.0,-0.11,-6.00,29.84,22.05,152.0,-0.21,0.65,33.87,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-42.18,46.0,M-SC,7.05,119977.0,5179.0,88867.0,-2.68,4.51,74.07,81.92,223.0,0.06,0.84,42.03,XR,NTT,DURABLES
46,KPIGREEN,731.05,4.03,46.0,H-SC,9.48,123971.0,-1326.0,60250.0,-2.24,-1.06,48.60,47.03,141.0,-0.02,0.87,55.22,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,71.35,61.0,H-SC,4.82,148417.0,-34358.0,145434.0,-1.02,-18.80,97.99,60.77,125.0,-0.24,1.04,33.66,XR,NTT,JEWELLERY
5,ANGELONE,3033.00,21.94,68.0,X-SC,6.45,215660.0,24654.0,33039.0,0.52,12.91,15.32,30.21,157.0,0.75,1.52,36.48,X40N,NTT,FINANCE
7,ASIANTILES,137.00,7394.44,64.0,L-SC,13.01,82204.0,-11606.0,88221.0,0.53,-12.37,107.32,81.67,269.0,-0.13,0.58,59.19,XR,NTT,CERAMICS
6,ASIANPAINT,3465.66,-8.88,71.0,X-LC,5.04,230014.0,-21754.0,74962.0,0.42,-8.64,32.59,21.13,27.0,-0.29,1.62,23.72,X40,ATH,PAINTS
68,SONACOMS,806.63,-31.14,68.0,M-SC,6.26,85750.0,-15409.0,55412.0,-0.22,-15.23,64.62,39.54,202.0,-0.28,0.60,20.92,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.84
1,25,44.57
2,50,75.56


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.67
LC    33.25
MC    23.12
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.97
X40      16.38
X40N     12.45
XY25     11.94
XR        9.80
AR        7.75
OX40N     6.58
X200      1.79
X5K       1.48
SR        1.03
MH        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.10
X-LC    22.53
X-MC    18.72
M-SC    12.54
X-SC     5.58
M-LC     5.17
H-LC     4.55
H-MC     2.32
M-MC     1.76
L-SC     1.45
L-LC     1.00
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.21,-4.70,38.55
IT,12.56,-17.81,80.87
FINANCE,12.17,-9.32,58.88
MISC,7.75,-18.14,77.74
PAINTS,5.70,-13.95,30.93
ELECTRICAL,5.68,-11.98,51.67
BANKS,3.98,-19.28,88.67
INSURANCE,3.94,1.11,35.77
AUTO,2.70,-49.59,112.02


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3315256.0,22
XR,1264199.0,13
AR,1185935.0,9
X40,829452.0,12
X40N,701248.0,9
OX40N,550196.0,9
XY25,476470.0,8
SR,274957.0,2
X5K,106855.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3407635.0,24
M-SC,1261057.0,15
X-MC,1213819.0,14
X-LC,1077945.0,13
X-SC,557579.0,6
H-LC,291356.0,3
M-LC,273924.0,4
H-MC,258697.0,2
L-SC,255053.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1185450.0      6
           AR         840260.0      5
           XR         792588.0      7
M-SC       XY24       738733.0      6
X-MC       XY24       576125.0      4
X-LC       X40        447310.0      5
           XY24       311907.0      2
X-SC       X40N       303887.0      4
X-MC       X40        295323.0      6
H-SC       SR         274957.0      2
           OX40N      254130.0      3
X-LC       X40N       237422.0      3
H-LC       AR         222073.0      2
H-MC       XY24       184095.0      1
X-MC       XY25       182432.0      2
X-SC       XY24       166873.0      1
L-SC       XR         164711.0      2
X-MC       X40N       159939.0      2
M-MC       XY24       142062.0      1
M-SC       XR         138292.0      2
           OX40N      131122.0      4
           XY25       129308.0      1
           AR         123602.0      2
M-LC       XR         111594.0      1
           X5K        106855.0      1
L-SC       OX40N       90342.0      1
X-SC       X40         86819.0      1
X-LC       XY25        81306.0      3
H-MC       OX40N       74602.0      1
H-LC       X200        69283.0      1
H-SC       MH          60250.0      1
L-MC       XR          57014.0      1
M-LC       XY25        45464.0      1
L-LC       XY25        37960.0      1
M-LC       XY24        10011.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 35.0 seconds
